In [ ]:
## Descargar kaggle.json de "account" en la cuenta de kaggle.

## Descargar librerías importantes

In [1]:
!pip install kaggle

In [2]:
!pip install ray

In [3]:
!pip install ray[data]

In [4]:
!pip install emoji

In [22]:
import pandas as pd
import numpy as np
import re
import string
import emoji
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# ML Libraries
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm, tree

import ray
from joblib import parallel_backend
import math, statistics, time
import numpy as np
import os, psutil

## Acceder a la base de datos

In [2]:
!cp kaggle.json kaggle

In [3]:
! chmod 600 kaggle.json

In [6]:
! cp kaggle.json ~/.kaggle/

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle competitions download -c fake-news

  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 16.3MB/s]
 76% 28.0M/37.0M [00:00<00:00, 58.5MB/s]
100% 37.0M/37.0M [00:00<00:00, 81.9MB/s]
 64% 6.00M/9.42M [00:00<00:00, 62.4MB/s]
100% 9.42M/9.42M [00:00<00:00, 60.2MB/s]


In [9]:
! unzip test.csv.zip
! unzip train.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


## Entorno Ray

In [6]:
num_workers = 8
trials = 20
ray.init(num_cpus=num_workers, ignore_reinit_error=True)

{'metrics_export_port': 46048,
 'node_id': 'e4338e65d4a758597b318fdc9ac81f008fb69443e74197a73b4aac95',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2021-11-29_00-30-26_903616_454/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2021-11-29_00-30-26_903616_454/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2021-11-29_00-30-26_903616_454',
 'webui_url': None}

## Lectura de datos

In [32]:
@ray.remote
def ray_read(dat):
    return pd.read_csv(dat)

In [35]:
start = time.time()
df_submit_ray = ray_read.remote("/content/submit.csv")
df_submit = ray.get(df_submit_ray)
duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

0.008511543273925781
255.80859375


In [44]:
start = time.time()
df_train_ray = ray_read.remote("/content/train.csv")
df_train = ray.get(df_train_ray)
duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

1.6012375354766846
1374.43359375


In [45]:
start = time.time()
df_test_ray = ray_read.remote("/content/test.csv")
df_test = ray.get(df_test_ray)
duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

0.43208885192871094
1425.72265625


In [46]:
df_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## Eliminar nulos

In [42]:
@ray.remote
def ray_remove_null(dat):
    return dat[dat.text.notna()]

In [47]:
start = time.time()
df_train_rm = ray_remove_null.remote(df_train)
df_train = ray.get(df_train_rm)
duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

1.301347255706787
1679.42578125


In [48]:
start = time.time()
df_test_rm = ray_remove_null.remote(df_test)
df_test = ray.get(df_test_rm)
duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

0.30404067039489746
1686.8203125


In [49]:
display(type(df_test))
display((df_test.count(), len(df_test.columns)))

pandas.core.frame.DataFrame

(id        5193
 title     5071
 author    4697
 text      5193
 dtype: int64, 4)

## Procesamiento de datos

In [50]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [51]:
stop_words = set(stopwords.words('english'))

In [52]:
def give_emoji_free_text(text): 
    return emoji.get_emoji_regexp().sub(r'', text)

In [53]:
def preprocess_tweet_text(tweet):
    emoticon_string = r"""
    (?:
    [<>]?
    [:;=8]                     
    [\-o\*\']?                 
    [\)\]\(\[dDpP/\:\}\{@\|\\]       
    |
    [\)\]\(\[dDpP/\:\}\{@\|\\]
    [\-o\*\']?                 
    [:;=8]                     
    [<>]?
    )"""

    replacements = (
    ("á", "a"),
    ("é", "e"),
    ("í", "i"),
    ("ó", "o"),
    ("ú", "u"),
    )

    tweet = tweet.lower()
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#','', tweet)
    # Remove user ¿ and ¡ from tweet
    tweet = re.sub(r'\¿|\¡|\?|\!','', tweet)
    # Remove punctuations
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # remove graphical emoji
    tweet = give_emoji_free_text(tweet) 
    # remove textual emoji
    tweet = re.sub(emoticon_string,'',tweet)
    #remove accent mark
    for a, b in replacements:
        tweet = tweet.replace(a, b)

    # Remove stopwords
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
    
    return " ".join(filtered_words)

In [54]:
def get_feature_vector(train_fit):
    vector = TfidfVectorizer(sublinear_tf=True)
    vector.fit(train_fit)
    return vector

In [55]:
df_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [56]:
@ray.remote
def ray_preprocess_tweet_text(tweet):
    return preprocess_tweet_text(tweet)

In [57]:
start = time.time()
df_train['text'] = df_train['text'].apply(ray_preprocess_tweet_text.remote)
df_train['text'] = df_train['text'].apply(ray.get)
duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

(pid=1085) [2021-11-29 01:31:00,993 C 1085 1141] core_worker.cc:194:  Check failed: core_worker_process The core worker process is not initialized yet or already shutdown.
(pid=1085) *** StackTrace Information ***
(pid=1085)     ray::SpdLogMessage::Flush()
(pid=1085)     ray::RayLog::~RayLog()
(pid=1085)     ray::core::CoreWorkerProcess::EnsureInitialized()
(pid=1085)     ray::core::CoreWorkerProcess::GetCoreWorker()
(pid=1085)     __pyx_pw_3ray_7_raylet_10CoreWorker_23get_worker_id()
(pid=1085)     _PyMethodDef_RawFastCallKeywords
(pid=1085) 


(ray_preprocess_tweet_text pid=1044) 
(ray_preprocess_tweet_text pid=995) 
(ray_preprocess_tweet_text pid=1035) 
(ray_preprocess_tweet_text pid=627) 
(ray_preprocess_tweet_text pid=1132) 
(ray_preprocess_tweet_text pid=1094) 
(ray_preprocess_tweet_text pid=626) 
(ray_preprocess_tweet_text pid=1094) 
(ray_preprocess_tweet_text pid=996) 
(ray_preprocess_tweet_text pid=627) 
(ray_preprocess_tweet_text pid=627) 
(ray_preprocess_tweet_text pid=1094) 
(ray_preprocess_tweet_text pid=1132) 
(ray_preprocess_tweet_text pid=1132) 
(ray_preprocess_tweet_text pid=1094) 
(ray_preprocess_tweet_text pid=996) 
(ray_preprocess_tweet_text pid=996) 
585.7289617061615
1807.83984375


In [58]:
df_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide ’ even see comey ’ letter jason...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,ever get feeling life circles roundabout rathe...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,truth might get fired october 29 2016 tension ...,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,videos 15 civilians killed single us airstrike...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,print iranian woman sentenced six years prison...,1


## Pre-procesamiento

In [60]:
@ray.remote
def ray_get_feature_vector(da):
    return get_feature_vector(da)

In [62]:
start = time.time()

tf_vector_ray = ray_get_feature_vector.remote(np.array(df_train.iloc[:, 3]).ravel())
tf_vector = ray.get(tf_vector_ray)
X_train = tf_vector.transform(np.array(df_train.iloc[:, 3]).ravel())
y_train = np.array(df_train.iloc[:, 4]).ravel()

duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

22.835763692855835
2073.19921875


In [67]:
df_submit.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [66]:
df_test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [ ]:
df_test = pd.merge(product,customer,on='Product_ID',how='left')

In [ ]:
@ray.remote
def join_test(bd1,bd2):
    return pd.merge(bd1, bd2, on="id",how='left')



In [ ]:
start = time.time()

tf_vector_ray = ray_get_feature_vector.remote(np.array(df_test.iloc[:, 3]).ravel())
tf_vector = ray.get(tf_vector_ray)
X_test = tf_vector.transform(np.array(df_test.iloc[:, 3]).ravel())
y_test = np.array(df_test.iloc[:, 4]).ravel()

duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

## Modelamiento

In [ ]:
from ray.util.joblib import register_ray


In [ ]:
register_ray()
NB_model = MultinomialNB()
with parallel_backend('ray'):
    NB_model.fit(X_train, y_train)
y_predict_nb = NB_model.predict(X_test)
print(accuracy_score(y_test, y_predict_nb))

0.7823260293763544


In [ ]:
print(classification_report(y_test , y_predict_nb))

              precision    recall  f1-score   support

           0       0.70      1.00      0.82      2068
           1       1.00      0.57      0.72      2085

    accuracy                           0.78      4153
   macro avg       0.85      0.78      0.77      4153
weighted avg       0.85      0.78      0.77      4153



In [ ]:
LR_model = LogisticRegression(solver='lbfgs')
with parallel_backend('ray'):
    LR_model.fit(X_train, y_train)
y_predict_lr = LR_model.predict(X_test)
print(accuracy_score(y_test, y_predict_lr))

2021-11-28 08:25:24,710	WARNING pool.py:470 -- The 'context' argument is not supported using ray. Please refer to the documentation for how to control ray initialization.


0.959306525403323


In [ ]:
print(classification_report(y_test , y_predict_lr))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96      2068
           1       0.95      0.97      0.96      2085

    accuracy                           0.96      4153
   macro avg       0.96      0.96      0.96      4153
weighted avg       0.96      0.96      0.96      4153



In [ ]:
@ray.remote
def ray_RandomForestClassifier():
    return RandomForestClassifier()

In [ ]:
RF_model = ray_RandomForestClassifier.remote()

In [ ]:
start = time.time()
RF_model = ray.get(ray_RandomForestClassifier.remote())
RF_model.fit(X_train, y_train)
y_predict_rf = RF_model.predict(X_test)
duration = time.time() - start
print(duration)
print(accuracy_score(y_test, y_predict_rf))

108.95050573348999
0.9227064772453648


In [ ]:
print(classification_report(y_test , y_predict_rf))

              precision    recall  f1-score   support

           0       0.90      0.95      0.92      2068
           1       0.94      0.90      0.92      2085

    accuracy                           0.92      4153
   macro avg       0.92      0.92      0.92      4153
weighted avg       0.92      0.92      0.92      4153



In [ ]:
SVM_model = svm.SVC(C=1.9, kernel='linear')
SVM_model.fit(X_train, y_train)
y_predict_svm = SVM_model.predict(X_test)
print(accuracy_score(y_test, y_predict_svm))

0.9703828557669155


In [ ]:
print(classification_report(y_test , y_predict_svm))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2068
           1       0.97      0.97      0.97      2085

    accuracy                           0.97      4153
   macro avg       0.97      0.97      0.97      4153
weighted avg       0.97      0.97      0.97      4153

